# 2. Mājasdarbs. Oļegs Korsaks 051RDB146
• Atrast datu kopu internetā. Piemēram no Awesomedata vai UCI Machine Learning

Repository vai citur

– Vēlams ar simtiem līdz tūkstošiem ierakstiem,

– Vēlams desmitiem atribūtiem,

– Skaitliskās vērtības atribūtos,

– Ar mērķa klasi/klasēm, bet var arī nepārtraukta vērtība,

– Dažām datu kopām var nebūt izvirzīta mērķa klase. Šādos gadījumos var izmantot vienu no atribūtiem par mērķa klasi (ja ir iespējams loģiski pamatot).

• Sadalīt datu kopu apmācības un testa kopās.

• Izveidot daudzslāņu mākslīgo neironu tīklu ar tensorflow un veikt apmācību.

• Aprēķināt un novērtēt apmācības un testa kopas kļūdu.

• Pamainīt tīkla konfigurāciju un novērtēt izmaiņu ietekmi

– Arhitektūru – slēpto slāņu skaits, neironu skaits slāņos,

– Aktivācijas funkcijas,

– Optimizācijas metodi.

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf

gpus = tf.config.list_physical_devices('GPU')

if gpus:
    # Restrict TensorFlow to only allocate 1GB of memory on the first GPU
    try:
        tf.config.experimental.set_memory_growth(gpus[0], True)

        # config = tf.compat.v1.ConfigProto()
        # config.gpu_options.per_process_gpu_memory_fraction = 0.2
        # config.gpu_options.allow_growth = True
        # session = tf.compat.v1.InteractiveSession(config=config)
        tf.config.set_logical_device_configuration(
            gpus[0],
            [tf.config.LogicalDeviceConfiguration(memory_limit=2500)]
        )
        logical_gpus = tf.config.list_logical_devices('GPU')
        print(f'{len(gpus)} Physical GPUs, {len(logical_gpus)} Logical GPUs')
    except RuntimeError as e:
        # Virtual devices must be set before GPUs have been initialized
        print(e)

sns.set_context('notebook', font_scale=2.0)
sns.set_style('darkgrid')

from imblearn.over_sampling import SMOTE
from matplotlib import pyplot as plt

%matplotlib inline

from sklearn.model_selection import train_test_split

## Datu ielāde

In [ ]:
data = pd.read_csv('data/winequality-white.csv', sep=';')

### Datu analīze

In [ ]:
data.head()

In [ ]:
data.info()

In [ ]:
data.describe().T

## Datu vizualizācija un analīze

Paskatīsimies, vai ir dati, kuru vērtības ļoti atšķiras no vairuma.

In [ ]:
fig, ax = plt.subplots(ncols=6, nrows=2, figsize=(20, 10))
index = 0
ax = ax.flatten()

for col, value in data.items():
    sns.boxplot(y=col, data=data, ax=ax[index])
    index += 1

plt.tight_layout(pad=0.5, w_pad=0.0)

Dažiem (piem. "total sulfur dioxide") ir tādas vērtības. Varbūt tie ietekmēs precizitāti.

Paskatīsimies uz vērtību sadali.

In [ ]:
fig, ax = plt.subplots(ncols=6, nrows=2, figsize=(20, 10))
index = 0
ax = ax.flatten()

for col, value in data.items():
    sns.histplot(value, ax=ax[index])
    index += 1

plt.tight_layout(pad=0.5, w_pad=0.0)

In [ ]:
sns.countplot(x=data['quality'])

Bet "quality" gan ir cita lieta - mums pārsvārā ir vērtības 5,6 un pat nav dažu vērtību vispār. Tāda klašu nelīdzsvarotība nozīmē, ka modelis varēs labi noklasificēt tikai 5. un 6. kvalitātes gadījumus. Tātad datus būs janobalansē.

### Paramtetru korelācija

In [ ]:
corr = data.corr()
plt.figure(figsize=(20, 10))
sns.heatmap(corr, annot=True, cmap='coolwarm')

Te ir redzams, ka "alcohol" ietekmē kvalitāti visvairāk, tātad tas ir svārīgs parametrs. Vēl alkohols korelē ar blīvumu (density), tas nozīmē, ka varam atstāt tikai vienu. Tas pats attiecas uz "free"/"total" sulfur dioxide, un dažu veidu skābumi (acidity). Ja būs problēmas ar precizitāti - varbūt noņemšu tos.

## Datu sadalīšana apmācības un pārbaudes kopās

In [ ]:
samples = data.copy()
# samples.pop('free sulfur dioxide')
# samples.pop('citric acid')
qualities = samples.pop('quality')

### Klašu vērtību balansēšana

In [ ]:
qualities.value_counts()

In [ ]:
oversample = SMOTE(k_neighbors=4)
samples, qualities = oversample.fit_resample(samples, qualities)

In [ ]:
qualities.value_counts()

Dati tika sabalansēti.

In [ ]:
# samples['fixed acidity'] = np.log(samples['fixed acidity'] + 0.01)
# samples['residual sugar'] = np.log(samples['residual sugar'] + 0.01)
# samples['free sulfur dioxide'] = np.log(samples['free sulfur dioxide'] + 0.01)
# samples['total sulfur dioxide'] = np.log(samples['total sulfur dioxide'] + 0.01)

tf.convert_to_tensor(samples)

In [ ]:
normalizer = tf.keras.layers.Normalization(axis=-1)
normalizer.adapt(samples)

Izveidosim modeli ar vienu slēpto slāni ar input_count neironiem un izejas slāni ar vienu neironu.

In [ ]:
def build_model(input_count: int, normalizer):
    init = tf.keras.initializers.TruncatedNormal(stddev=0.01, seed=42)
    model = tf.keras.models.Sequential([
        normalizer,
        tf.keras.layers.Dense(input_count, kernel_initializer=init, activation='relu', input_shape=(input_count,)),
        tf.keras.layers.Dense(1)
    ])

    adam = tf.keras.optimizers.Adam(learning_rate=0.0005)

    model.compile(loss='mse', optimizer=adam, metrics=['mse'])

    return model

In [ ]:
attr_count = len(samples.keys())
print(attr_count, len(qualities.value_counts()))

Mums ir 11 atribūti un 7 klases, jo nav datu priekš citām klasēm.
Tagad uzbūvēsim modeli un trenēsim to:

In [ ]:
def plot_history(history):
    plt.figure(figsize=(18, 10))
    plt.plot(history.history['mse'])
    plt.plot(history.history['val_mse'])
    plt.title('model_mse')
    plt.ylabel('error')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='best')
    plt.show()

In [ ]:
def evaluate_model(model, samples, qualities, epochs=100):
    samples_train, samples_test, qualities_train, qualities_test = train_test_split(samples, qualities, test_size=0.3,
                                                                                    random_state=0)
    history = model.fit(samples_train, qualities_train, validation_data=(samples_test, qualities_test), shuffle=False,
                        epochs=epochs, verbose=0)
    plot_history(history)
    result = model.predict(samples_test)

    for sample_idx in (-5, -10, -20):
        print(f'Predicted: {result[sample_idx][0]}, Expected: {qualities_test.iat[sample_idx]}')

In [ ]:
model = build_model(attr_count, normalizer)
evaluate_model(model, samples, qualities, epochs=100)

Nav tik labi rezultāti. Pievienosim vēl divus tik pat lielus slāņus.

In [ ]:
def build_model(input_count: int, normalizer):
    init = tf.keras.initializers.TruncatedNormal(stddev=0.01, seed=42)
    model = tf.keras.models.Sequential([
        normalizer,
        tf.keras.layers.Dense(input_count, kernel_initializer=init, activation='relu', input_shape=(input_count,)),
        tf.keras.layers.Dense(input_count, activation='relu'),
        tf.keras.layers.Dense(input_count, activation='relu'),
        tf.keras.layers.Dense(1)
    ])

    adam = tf.keras.optimizers.Adam(learning_rate=0.0005)

    model.compile(loss='mse', optimizer=adam, metrics=['mse'])

    return model

In [ ]:
model = build_model(attr_count, normalizer)
evaluate_model(model, samples, qualities, epochs=100)

Pamainīsim neironu skaitu slāņos.

In [ ]:
def build_model(input_count: int, normalizer):
    init = tf.keras.initializers.TruncatedNormal(stddev=0.01, seed=42)
    model = tf.keras.models.Sequential([
        normalizer,
        tf.keras.layers.Dense(32, kernel_initializer=init, activation='relu', input_shape=(input_count,)),
        tf.keras.layers.Dense(16, activation='relu'),
        tf.keras.layers.Dense(8, activation='relu'),
        tf.keras.layers.Dense(1)
    ])

    adam = tf.keras.optimizers.Adam(learning_rate=0.0005)

    model.compile(loss='mse', optimizer=adam, metrics=['mse'])

    return model

In [ ]:
model = build_model(attr_count, normalizer)
evaluate_model(model, samples, qualities, epochs=100)

Pamēģināšu to pašu modeli bet ar vairākām epohām, 1000.

In [ ]:
model = build_model(attr_count, normalizer)
evaluate_model(model, samples, qualities, epochs=1000)

Palielināšu learning rate x10.

In [ ]:
def build_model(input_count: int, normalizer):
    init = tf.keras.initializers.TruncatedNormal(stddev=0.01, seed=42)
    model = tf.keras.models.Sequential([
        normalizer,
        tf.keras.layers.Dense(32, kernel_initializer=init, activation='relu', input_shape=(input_count,)),
        tf.keras.layers.Dense(16, activation='relu'),
        tf.keras.layers.Dense(8, activation='relu'),
        tf.keras.layers.Dense(1)
    ])

    adam = tf.keras.optimizers.Adam(learning_rate=0.005)

    model.compile(loss='mse', optimizer=adam, metrics=['mse'])

    return model

In [ ]:
model = build_model(attr_count, normalizer)
evaluate_model(model, samples, qualities, epochs=1000)

Tagad kļūda raustas. Samazināšu learning rate x2. un Palielināšu epohu skaitu x2.

In [ ]:
def build_model(input_count: int, normalizer):
    init = tf.keras.initializers.TruncatedNormal(stddev=0.01, seed=42)
    model = tf.keras.models.Sequential([
        normalizer,
        tf.keras.layers.Dense(32, kernel_initializer=init, activation='relu', input_shape=(input_count,)),
        tf.keras.layers.Dense(16, activation='relu'),
        tf.keras.layers.Dense(8, activation='relu'),
        tf.keras.layers.Dense(1)
    ])

    adam = tf.keras.optimizers.Adam(learning_rate=0.00025)

    model.compile(loss='mse', optimizer=adam, metrics=['mse'])

    return model

In [ ]:
model = build_model(attr_count, normalizer)
evaluate_model(model, samples, qualities, epochs=2000)

Izmēģināšu mean absolute erorr loss-funkciju.

In [ ]:
def build_model(input_count: int, output_count: int):
    init = tf.keras.initializers.TruncatedNormal(stddev=0.01, seed=42)
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Dense(64, kernel_initializer=init, activation='relu', input_shape=(input_count,)))
    model.add(tf.keras.layers.Dense(32, activation='relu'))
    model.add(tf.keras.layers.Dense(16, activation='relu'))
    model.add(tf.keras.layers.Dense(1))

    adam = tf.keras.optimizers.Adam(learning_rate=0.0025)

    model.compile(loss='mae', optimizer=adam, metrics=['mse'])

    return model

In [ ]:
model = build_model(attr_count, 1)
evaluate_model(model, samples, qualities, epochs=2000)

Palika sliktāk. Pamēģināšu Nadam optimizācijas algoritmu.

In [ ]:
def build_model(input_count: int, output_count: int):
    init = tf.keras.initializers.TruncatedNormal(stddev=0.01, seed=42)
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Dense(64, kernel_initializer=init, activation='relu', input_shape=(input_count,)))
    model.add(tf.keras.layers.Dense(32, activation='relu'))
    model.add(tf.keras.layers.Dense(16, activation='relu'))
    model.add(tf.keras.layers.Dense(1))

    adam = tf.keras.optimizers.Nadam(learning_rate=0.0025)

    model.compile(loss='mse', optimizer=adam, metrics=['mse'])

    return model

In [ ]:
model = build_model(attr_count, 1)
evaluate_model(model, samples, qualities, epochs=2000)

Nu ļoti slikts rezultāts :/

Pamēģināšu palielināt slāņu, neironu un epohu skaitu.

In [ ]:
def build_model(input_count: int, output_count: int):
    init = tf.keras.initializers.TruncatedNormal(stddev=0.01, seed=42)
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Dense(64, kernel_initializer=init, activation='relu', input_shape=(input_count,)))
    model.add(tf.keras.layers.Dense(32, activation='relu'))
    model.add(tf.keras.layers.Dense(16, activation='relu'))
    model.add(tf.keras.layers.Dense(8, activation='relu'))
    model.add(tf.keras.layers.Dense(4, activation='relu'))
    model.add(tf.keras.layers.Dense(1))

    adam = tf.keras.optimizers.Adam(learning_rate=0.001)

    model.compile(loss='mse', optimizer=adam, metrics=['mse'])

    return model

In [ ]:
model = build_model(attr_count, 1)
evaluate_model(model, samples, qualities, epochs=10000)

Varbūt aktivācijas funkcija ir pie vainas (neticu, bet jāparbauda). Pamainīšu uz sigmoid.

In [ ]:
def build_model(input_count: int, output_count: int):
    init = tf.keras.initializers.TruncatedNormal(stddev=0.01, seed=42)
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Dense(64, kernel_initializer=init, activation='sigmoid', input_shape=(input_count,)))
    model.add(tf.keras.layers.Dense(32, activation='sigmoid'))
    model.add(tf.keras.layers.Dense(16, activation='sigmoid'))
    model.add(tf.keras.layers.Dense(8, activation='sigmoid'))
    model.add(tf.keras.layers.Dense(4, activation='sigmoid'))
    model.add(tf.keras.layers.Dense(1))

    adam = tf.keras.optimizers.Adam(learning_rate=0.001)

    model.compile(loss='mse', optimizer=adam, metrics=['mse'])

    return model

In [ ]:
model = build_model(attr_count, 1)
evaluate_model(model, samples, qualities, epochs=10000)

Bet ja es pamainīšu pēdēja neirona aktivācijas funkciju uz relu? Jo pirms tam tam bija lineāra funkcija.

In [ ]:
def build_model(input_count: int, output_count: int):
    init = tf.keras.initializers.TruncatedNormal(stddev=0.01, seed=42)
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Dense(64, kernel_initializer=init, activation='relu', input_shape=(input_count,)))
    model.add(tf.keras.layers.Dense(32, activation='relu'))
    model.add(tf.keras.layers.Dense(16, activation='relu'))
    model.add(tf.keras.layers.Dense(8, activation='relu'))
    model.add(tf.keras.layers.Dense(4, activation='relu'))
    model.add(tf.keras.layers.Dense(1, activation='relu'))

    adam = tf.keras.optimizers.Adam(learning_rate=0.001)

    model.compile(loss='mse', optimizer=adam, metrics=['mse'])

    return model

In [ ]:
model = build_model(attr_count, 1)
evaluate_model(model, samples, qualities, epochs=10000)